Importing:

In [6]:
from sklearn.model_selection import StratifiedKFold
import torch
import json
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU for training.")
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU for training.")

Using GPU for training.


In [8]:
%run "../DL_notebooks/src/dataloading.ipynb"
%run "../DL_notebooks/src/training.ipynb"
%run "../DL_notebooks/src/evaluate.ipynb"
%run "../DL_notebooks/src/visualizations.ipynb"
%run "../DL_notebooks/model_architectures.ipynb"

Data loaded and split into training, validation and testing datasets


Testing:

In [11]:
with open('../DL_notebooks/configs/model_configs.json', 'r') as model_config_file:
    model_configs = json.load(model_config_file)

models = []
model_architectures = []
for config in model_configs:
    model_architecture = config['model-architecture']
    epochs = config['epoch-count']
    optimizer_lr = config['optimizer-lr']
    model_architectures.append(model_architecture)

    match model_architecture:
        case 'CNN':
            model = MRI_CNN(**config['cnn-model-hyperparameters'])
            model = model.to(device)
        case 'ResNet':
            num_classes = config['resnet-model-hyperparameters']['num_classes']
            layers = config['resnet-model-hyperparameters']['layers']
            model = MRI_ResNet(BasicBlock, layers, num_classes)
            model = model.to(device)
        case 'ResNet2':
            num_classes2 = config['resnet2-model-hyperparameters']['num_classes']
            model = MRI_ResNet2(num_classes2)
            model = model.to(device)
            match config['optimizer']:
                case 'adamW':
                    optimizer = optim.AdamW(model.parameters(), lr=optimizer_lr)
                case _:
                    optimizer = optim.Adam(model.parameters(), lr=optimizer_lr)
        case 'AlexNet':
            num_classes = config['alexnet-model-hyperparameters']['num_classes']
            model = MRI_AlexNet(num_classes)
            model = model.to(device)

    match config['loss-function']:
        case 'cross-entropy':
            criterion = nn.CrossEntropyLoss()
        case _:
            criterion = nn.CrossEntropyLoss()

    if config['optimizer'] == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=optimizer_lr)
    else:
        optimizer = optim.Adam(model.parameters(), lr=optimizer_lr)

    
    models.append((model, criterion, optimizer))

In [12]:
num_splits = 5

stratified_kf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)
model_results = []
for model, criterion, optimizer in models:
    fold_results = []
    train_loader_fold = train_loader
    val_loader_fold = val_loader
    val_accuracies = []
    training_accuracies = []
    count = 0
    for train_index, val_index in stratified_kf.split(train_labels, train_labels):

        for epoch in tqdm(range(epochs), desc="Training", unit="epoch"):
            train_loss, train_accuracy, _, _ = train(model, train_loader=train_loader_fold, optimizer=optimizer, criterion=criterion, device=device)
            training_accuracies.append(train_accuracy)
            val_loss, val_accuracy, val_labels, val_preds = evaluate(model, val_loader_fold, criterion=criterion, device=device)
            fold_results.append({
                "train_loss": train_loss,
                "train_accuracy": train_accuracy,
                "val_loss": val_loss,
                "val_accuracy": val_accuracy,
            })
            val_accuracies.append(val_accuracy)
        
    plt.figure(figsize=(10, 5))
    plt.plot(training_accuracies, label="Training Accuracy")
    plt.plot(val_accuracies, label="Validation Accuracy")
    plt.title(f"Training and Validation Accuracy Over Time for {model_architectures[count]}")
    plt.xlabel("Fold")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()
    avg_val_accuracy = sum(val_accuracies) / len(val_accuracies)
    model_results.append((model, avg_val_accuracy))
    count = count + 1


    
    

Training:  15%|█▌        | 3/20 [01:38<08:58, 31.66s/epoch]

In [ ]:
test_losses = []
test_accuracies = []
best_model = max(model_results, key=lambda x: x[1]["val_accuracy"])
best_model, best_metrics = best_model


print(f"Average Validation Accuracy: {best_metrics['val_accuracy']:.4f}")
print(f"Average Validation Loss: {best_metrics['val_loss']:.4f}")

In [ ]:
for epoch in tqdm(range(epochs), desc="Testing", unit="epoch"):
    start_time = time.time()
    test_loss, test_accuracy, test_labels, test_preds = evaluate(model, test_loader, criterion=criterion, device=device)
    end_time = time.time()

    epoch_time = end_time - start_time

    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

    tqdm.write(
        f"Testing Epoch [{epoch + 1}/{epochs}] "
        f"Test Loss: {test_loss:.4f} "
        f"Test Accuracy: {test_accuracy * 100:.2f}% "
        f"Time per Epoch: {epoch_time:.2f} seconds"
    )

avg_test_loss = sum(test_losses) / len(test_losses)
avg_test_accuracy = sum(test_accuracies) / len(test_accuracies)
print(f"Average Test Loss: {avg_test_loss:.4f}")
print(f"Average Test Accuracy: {avg_test_accuracy * 100:.2f}%")

In [ ]:
for model_name, accuracies in validation_accuracies.items():
    plt.plot(range(epochs), accuracies, label=model_name)

plt.title("Validation Accuracies Over Time")
plt.xlabel("Epoch")
plt.ylabel("Validation Accuracy")
plt.legend()
plt.show()